# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# ML
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

# NLP libraries
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)
X = df.message
y = df.iloc[:,4:]

In [9]:
X[:5]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [11]:
y[:5]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    # remove numbers, punctuation and tokenize text
    
    tokens = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
    
    # lemmatization 
    lemma = WordNetLemmatizer()
    # save cleaned tokens
    clean_tokens = [lemma.lemmatize(tok).strip() for tok in tokens]
    
    # remove stopwords
    english_stopwords = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in english_stopwords]
    
    return clean_tokens
    
    

In [13]:
d = "Mary had a little Lamp's"

In [14]:
tokenize(d)

['mary', 'little', 'lamp']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
  

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
def classification_rep_col(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('**********',col,'*********')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

In [19]:

y_pred = pipeline.predict(X_test)


In [20]:
classification_rep_col(y_test, y_pred)

********** related *********
             precision    recall  f1-score   support

          0       0.63      0.49      0.55      1549
          1       0.85      0.91      0.88      4956
          2       0.48      0.20      0.29        49

avg / total       0.79      0.81      0.80      6554

********** request *********
             precision    recall  f1-score   support

          0       0.90      0.98      0.93      5434
          1       0.80      0.44      0.57      1120

avg / total       0.88      0.89      0.87      6554

********** offer *********
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

********** aid_related *********
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3868
          1       0.73      0.59      0.66      2686

avg / total       0.74 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

grid = GridSearchCV(pipeline, parameters)

In [22]:
grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)

best params by grid search 

In [23]:
print(grid.best_params_)

{'clf__estimator__min_samples_split': 2, 'tfidf__norm': 'l1'}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:

accuracy_model = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy of the tuned model :', accuracy_model)
classification_rep_col(y_test, y_pred)

accuracy of the tuned model : related                   0.807293
request                   0.884803
offer                     0.995880
aid_related               0.745499
medical_help              0.926610
medical_products          0.956820
search_and_rescue         0.973756
security                  0.978639
military                  0.968264
child_alone               1.000000
water                     0.946597
food                      0.941257
shelter                   0.934391
clothing                  0.986421
money                     0.976961
missing_people            0.988709
refugees                  0.969942
death                     0.957888
other_aid                 0.863747
infrastructure_related    0.932408
transport                 0.958651
buildings                 0.953006
electricity               0.980317
tools                     0.993897
hospitals                 0.989472
shops                     0.995728
aid_centers               0.987183
other_infrastructure     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### trying another classifier *one vs. rest*

In [28]:
pipeline_one_vs_rest = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
    ])

In [29]:
pipeline_one_vs_rest.fit(X_train, y_train)
y_pred = pipeline_one_vs_rest.predict(X_test)
classification_rep_col(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


********** related *********
             precision    recall  f1-score   support

          0       0.65      0.54      0.59      1549
          1       0.86      0.91      0.88      4956
          2       0.62      0.31      0.41        49

avg / total       0.81      0.82      0.81      6554

********** request *********
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5434
          1       0.76      0.59      0.66      1120

avg / total       0.89      0.90      0.89      6554

********** offer *********
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

********** aid_related *********
             precision    recall  f1-score   support

          0       0.79      0.81      0.80      3868
          1       0.72      0.68      0.70      2686

avg / total       0.76 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##### one vs. rest classifier performs sligtly better but for the excercise, I have shared the 'random forest' model 

### 9. Export your model as a pickle file

In [32]:
pickle.dump(pipeline, open('random_forest_model.pkl', 'wb'))

In [33]:
pickle.dump(pipeline_one_vs_rest, open('one_vs_rest.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.